## Documentation

To read more about the update API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update.html).

![update_docs](../images/update_docs.png)

## Connect to ElasticSearch

In [2]:
from pprint import pprint
from elasticsearch import Elasticsearch

HOST = "http://localhost:9200"

es = Elasticsearch(HOST)
client_info = es.info()
print("Connected tp Elasticsearch!")
pprint(client_info.body)

Connected tp Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'IzAz_bJfQnS_zfMDjIPmJA',
 'name': 'eb6cd056e782',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-01-09T14:09:01.578835424Z',
             'build_flavor': 'default',
             'build_hash': '0f88dde84795b30ca0d2c0c4796643ec5938aeb5',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '8.11.3',
             'minimum_index_compatibility_version': '6.0.0-beta1',
             'minimum_wire_compatibility_version': '6.8.0',
             'number': '7.17.27'}}


/tmp/ipykernel_14840/13932774.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client_info = es.info()


## Index documents

In [3]:
INDEX = "my_index"

settings = {
    "index": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}

es.indices.delete(index=INDEX, ignore_unavailable=True)
es.indices.create(index=INDEX, settings=settings)

/tmp/ipykernel_14840/2757242129.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=INDEX, ignore_unavailable=True)
/tmp/ipykernel_14840/2757242129.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=INDEX, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the `document_ids` list.

In [4]:
import json
from tqdm import tqdm

document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index=INDEX, body=document)
    document_ids.append(response["_id"])

  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_14840/2172172568.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=INDEX, body=document)
100%|██████████| 3/3 [00:00<00:00, 28.24it/s]


## Update API

### 1. If documents exists in the index

#### 1.1 Update an existing field

In [5]:
from pprint import pprint


response = es.update(
    index=INDEX,
    id=document_ids[0],
    script={
        "source": "ctx._source.title = params.title",
        "params": {
            "title": "New Title"
        }
    },
)
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_type': '_doc',
 '_version': 2,
 'result': 'updated'}


/tmp/ipykernel_14840/3847238007.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.update(


In [6]:
response = es.get(index=INDEX, id=document_ids[0])
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_type': '_doc',
 '_version': 2,
 'found': True}


/tmp/ipykernel_14840/132183440.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.get(index=INDEX, id=document_ids[0])


#### 1.2 Add a new field

To add a new field, you can either use the `script` argument or the `doc` argument.

##### 1.2.1 Method 1 (Script)

In [7]:
response = es.update(
    index=INDEX,
    id=document_ids[0],
    script={
        "source": "ctx._source.new_field = 'dummy_value'",
    },
)
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 4,
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_type': '_doc',
 '_version': 3,
 'result': 'updated'}


/tmp/ipykernel_14840/2502928623.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.update(


In [8]:
response = es.get(index=INDEX, id=document_ids[0])
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 4,
 '_source': {'created_on': '2024-09-22',
             'new_field': 'dummy_value',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_type': '_doc',
 '_version': 3,
 'found': True}


/tmp/ipykernel_14840/132183440.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.get(index=INDEX, id=document_ids[0])


##### 1.2.2 Method 2 (doc)

In [9]:
response = es.update(
    index=INDEX,
    id=document_ids[0],
    doc={
        "new_value_2": "dummy_value_2",
    },
)
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 5,
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_type': '_doc',
 '_version': 4,
 'result': 'updated'}


/tmp/ipykernel_14840/2052163754.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.update(


In [10]:
response = es.get(index='my_index', id=document_ids[0])
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 5,
 '_source': {'created_on': '2024-09-22',
             'new_field': 'dummy_value',
             'new_value_2': 'dummy_value_2',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_type': '_doc',
 '_version': 4,
 'found': True}


/tmp/ipykernel_14840/997944435.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.get(index='my_index', id=document_ids[0])


#### 1.3 Remove a field

In [11]:
response = es.update(
    index=INDEX,
    id=document_ids[0],
    script={
        "source": "ctx._source.remove('new_field')",
    },
)
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 6,
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_type': '_doc',
 '_version': 5,
 'result': 'updated'}


/tmp/ipykernel_14840/1487558570.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.update(


In [12]:
response = es.get(index=INDEX, id=document_ids[0])
pprint(response.body)

{'_id': 'NXgrJJUBpQvCJGK52k2y',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 6,
 '_source': {'created_on': '2024-09-22',
             'new_value_2': 'dummy_value_2',
             'text': 'This is the first sample document text.',
             'title': 'New Title'},
 '_type': '_doc',
 '_version': 5,
 'found': True}


/tmp/ipykernel_14840/132183440.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.get(index=INDEX, id=document_ids[0])


### 2. If documents doesn't exist in the index

We use `doc_as_upsert` to tell Elasticsearch that if the document does not exist, it should be inserted as a new document.

In [13]:
response = es.update(
    index=INDEX,
    id="1",
    doc={
        "book_id": 1234,
        "book_name": "A book",
    },
    doc_as_upsert=True,
)

/tmp/ipykernel_14840/2486154538.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.update(


In [14]:
pprint(response.body)

{'_id': '1',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 7,
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_type': '_doc',
 '_version': 1,
 'result': 'created'}


Sure enough, we have 4 documents now instead of 3.

In [16]:
response = es.count(index=INDEX)

response["count"]

/tmp/ipykernel_14840/1171723105.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.count(index=INDEX)


4